In [1]:
'''
This notebook should be run first: it creates a database with all covid-related articles accessible from pubmed 
since Jan 2020

'''

'\nThis notebook should be run first: it creates a database with all covid-related articles listed in pubmed.\n'

In [ ]:
import requests
from pymed import PubMed
import pickle
import time
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import random

In [6]:
def unique_authors(lst_authors):
    clean_list_authors = []
    for dicto in lst_authors:
        for elem in ['affiliation','firstname','initials','lastname']:
            if dicto[elem] == None:
                dicto[elem] = ' '
                
        unique_id = dicto['initials'] + ' ' + dicto['firstname']+ ' ' +dicto['lastname'] + ' - ' + dicto['affiliation']
        clean_list_authors.append(unique_id)
    return clean_list_authors


In [4]:
### just to test out small queries

pubmed = PubMed(tool="MyTool", email="my@email.address")
query = "covid[Title/Abstract] 2020/01/01[Date - Create] : 2020/12/31[Date - Create])"
# query formulation tool : https://pubmed.ncbi.nlm.nih.gov/advanced/
results = pubmed.query(query, max_results=10)

for article in results:
    print(article.keywords)

[]
['COVID-19', 'Disinfection', 'Hospital-acquired infections', 'Pulsed light', 'SARS-CoV-2', 'Touching surfaces']
['COVID-19', 'GRANCH', 'Graph theory', 'Hotspot', 'Mutation', 'SARS-CoV-2', 'Statistical data', 'Structural protein']
[]
['COVID-19', 'Domestic violence against women', 'Pandemic']
['COVID-19 pandemic', 'Child externalizing behaviors', 'Coparenting', 'Neurodevelopmental disorders', 'Parental stress']
['COVID-19', 'Central nervous system', 'Central retinal artery occlusion', 'Cerebrovascular disease', 'SARS-CoV-2']
['COVID-19', 'Coronavirus', 'Ischemic stroke', 'Large vessel occlusion', 'NIHSS']
['Acute kidney injury', 'Covid-19', 'Diabetes', 'Haematuria', 'Proteinuria', 'Urine examination']
['Anxiety', 'COVID-19', 'Depression', 'Diaphragmatic breathing relaxation training', 'Nurses', 'Sleep quality']


In [5]:
dict_authors = {}
dict_articles = {}

dates_lst = [('2020/01/01','2020/02/29'),('2020/03/01','2020/05/31'),('2020/06/01','2020/07/31'),
             ('2020/08/01','2020/09/30'),('2020/10/01','2020/11/31'),('2020/12/01','2021/01/31'),
             ('2021/02/01','2021/02/28'),('2021/03/01','2021/04/30'),('2021/05/01','2021/05/30')]

for chunk in dates_lst:
    pubmed = PubMed(tool="MyTool", email="my@email.address")
    query = f"covid[Title/Abstract] ({chunk[0]}[Date - Create] : {chunk[1]}[Date - Create])"
    # query formulation tool : https://pubmed.ncbi.nlm.nih.gov/advanced/
    results = pubmed.query(query, max_results=50000)

    for article in results:
        try:
            authors = unique_authors(article.authors)
            for author in authors:
                if author not in dict_authors.keys():
                    dict_authors[author] = {'Name':author.split(' - ')[0],'Articles':[article.doi],
                                            'ORCID_ID':'','Affiliation':author.split(' - ')[1],'email':''}
                else:
                    dict_authors[author]['Articles'].append([article.pubmed_id])

            if article.doi not in dict_articles.keys():
                dict_articles[article.doi] = {'Title':'',
                                              'Abstract':'',
                                              'Keywords':[],
                                              'First_Author':'',
                                              'Other_Authors':[],
                                              'Journal':'',
                                              'Publication_Date':''}
                if article.title:
                    dict_articles[article.doi]['Title'] = article.title
                if article.abstract:
                    dict_articles[article.doi]['Abstract'] = article.abstract
                if article.keywords:
                    for keyword in article.keywords:
                        dict_articles[article.doi]['Keywords'].append(keyword.lower().strip())
                if authors:
                    dict_articles[article.doi]['First_Author'] = authors[0]
                    dict_articles[article.doi]['Other_Authors'] = authors[1:]
                if article.publication_date:
                    dict_articles[article.doi]['Publication_Date'] = article.publication_date
                if article.journal:
                    dict_articles[article.doi]['Journal'] = article.journal
        except Exception as e:
            print(e)
                
    print("Finished for dates ", chunk, 'len dict authors', len(dict_authors), 'len dict articles', len(dict_articles))
    time.sleep(10)

None COVID-19: another infectious disease emerging at the animal-human interface.
None Neuropathic pain in adults: pharmacological management in non-specialist settings
Finished for dates  ('2020/01/01', '2020/02/29') len dict authors 949 len dict articles 178
None Comparative analysis of 
None COVID-19 and Congenital Heart Disease: Perspectives From a Resource-limited Setting.
None Taste and smell as chemosensory dysfunctions in COVID-19 infection.
None Transmission routes of SARS-CoV-2 and protective measures in dental clinics during the COVID-19 pandemic.
10.1056/NEJMc2017362 Screening for Covid-19 in Skilled Nursing Facilities. Reply.
10.1056/NEJMc2017362 Screening for Covid-19 in Skilled Nursing Facilities.
10.1056/NEJMc2017362 Screening for Covid-19 in Skilled Nursing Facilities.
10.1056/NEJMc2017362 Screening for Covid-19 in Skilled Nursing Facilities.
None Coronavirus (COVID-19): Let's Prevent Not Panic.
None Wuhan Coronavirus (Covid-19): Yet Another Bat Virus Zoonotic.
None Wo

None Is the Coronavirus (COVID-19) Pandemic an Indication to Temporarily Modify Dermatological Management Plans?
None [Renin-angiotensin-aldosterone blockers and Covic-19 infection : friends or enemies ?]
None [Brug af hydroxychloroquin til COVID-19].
None [De første kliniske data fra RASi-behandlede indlagte patienter med COVID-19 kommer nu, og vi skal ikke ændre medicinering].
None [Lack of clinical evidence for the use of hydroxychloroquine to treat SARS-CoV-2 infection].
None [Kronisk syge og udsatte gruppers situation under COVID-19-pandemien].
None [Anosmia and ageusia as primary symptoms of COVID-19].
None [Atypical clinical picture of COVID-19 in older patients].
None [Chest pain and dyspnea during the recovery period of COVID-19 pneumonia].
None [Dealing with COVID-19-associated coagulopathy].
None [Rational use of respiratory protective equipment: advice for health care professionals in time of COVID-19].
None [Ethical principles compromised during the COVID-19 pandemic?]
Non

10.1056/NEJMc2009666 Allocating Medical Resources in the Time of Covid-19.
10.1056/NEJMc2009666 Allocating Medical Resources in the Time of Covid-19.
10.1056/NEJMc2009666 Allocating Medical Resources in the Time of Covid-19.
10.1056/NEJMc2009666 Allocating Medical Resources in the Time of Covid-19.
None Pandemic Uncertainty: Considerations for Nephrology Nurses.
None Caring for Nephrology Patients and Staff During the COVID-19 Pandemic: Experiences of the Northwest Kidney Centers.
None Upper respiratory tract sampling in COVID-19.
None Diagnostic performance of COVID-19 serology assays.
None Challenges of Covid-19 testing.
None Use of free CAD software for 3D printing individualized face masks based on face scans.
None Does a Crying Child Enhance the Risk for COVID-19 Transmission?
None Coronavirus Disease (COVID-19) in Children: Indian Perspectives.
None Do Not Neglect the Children: Considerations for COVID-19 Pandemic.
None Coronavirus Disease (COVID-19) With Relevance to Pediatrics.

None Covid 19 pandemic and gynaecological laparoscopic surgery: knowns and unknowns.
None Laboratory Parameters in Detection of COVID-19 Patients with Positive RT-PCR; a Diagnostic Accuracy Study.
None Epidemiological and Clinical Aspects of COVID-19; a Narrative Review.
None Metronidazole; a Potential Novel Addition to the COVID-19 Treatment Regimen.
None Daily Situation Report on Coronavirus disease (COVID-19) in Iran; March 25, 2020.
None Daily Situation Report on Coronavirus disease (COVID-19) in Iran; March 23, 2020.
None Daily Situation Report on Coronavirus disease (COVID-19) in Iran; March 22, 2020.
None Daily Situation Report on Coronavirus disease (COVID-19) in Iran; March 17, 2020.
None Daily Situation Report on Coronavirus disease (COVID-19) in Iran; March 16, 2020.
None Daily Situation Report on Coronavirus disease (COVID-19) in Iran; March 15, 2020.
None Daily Situation Report on Coronavirus disease (COVID-19) in Iran; March 14, 2020.
None Radiographic and Clinical Featur

None Impact of COVID-19 on Nephrology Patients: A Mechanistic Outlook for Pathogenesis of Acute Kidney Injury.
None Biochemical Dynamics and Clinical Features of Novel Corona Virus (2019-nCoV).
None MMR Vaccine and Covid-19: A Myth or a Low Risk-High Reward Preventive Measure?
None Lung Ultrasound in COVID-19.
None Pediatric Coronavirus Disease-19 (COVID-19): Meta-analyzing Literature Versus Natural History.
None Managing Children with Special Needs in COVID-19 Times.
None Medical Education During the COVID-19 Pandemics - Challenges Ahead.
None COVID-19: Crisis Management in Lung Cancer Surgery.
None The impact of COVID-19 pandemic in the management of a Vascular Surgery Department.
None Global impact of the COVID 19 Pandemic in Portuguese Thoracic Surgery Centres.
None Cardiac Surgeons against the COVID-19 Pandemic.
None Apical Takotsubo Cardiomyopathy in a COVID-19 Patient Presenting with Stroke: A Case Report and Pathophysiologic Insights.
None Predicting Mechanical Ventilation Requ

...

None Neem (Azadirachta Indica A. Juss) Capsules for Prophylaxis of COVID-19 Infection: A Pilot, Double-Blind, Randomized Controlled Trial.
None Aggravated Respiratory Failure From COVID-19 Infection: Patient Care Management From Nurses in the Intensive Care Unit.
None Spotlight on Nursing: Hawai'i Nurses Play Major Role in COVID-19 Pandemic Response.
None Insights in Public Health: Pre-Surgical Screening and Incidence of COVID-19 Infection at the Shriners Hospitals for Children - Honolulu.
None Dental team and infection control for the COVID-19: (A short review of the current guidelines).
None COVID-19 shares clinical features with anti-melanoma differentiation-associated protein 5 positive dermatomyositis and adult Still's disease.
None Chemo-Preventive Effect of Vegetables and Fruits Consumption on the COVID-19 Pandemic.
None [Fragility in a Public Health perspective: principles and tools for a &quot;life course&quot; approach prevention- oriented].
None Challenges in Chronic Genetic

None Counselling patients for return to work on immunosuppression: practices of Canadian specialists during the COVID-19 pandemic.
None Do patients with rheumatoid arthritis show a different course of COVID-19 compared to patients with spondyloarthritis?
None High incidence of giant cell arteritis during the COVID-19 pandemic: no causal relationship but possible involvement of stress.
None Rapid quantification of COVID-19 pneumonia burden from computed tomography with convolutional LSTM networks.
None Osteoporosis management during the COVID-19 pandemic - Position paper.
None Symptomatic arterial thrombosis associated with novel coronavirus disease 2019 (COVID-19): report of two cases.
None Implementing an Interprofessional Anti-Racism Training With Community Partners During a Pandemic: Outcomes and Recommended Strategies.
None Lead Poisoning and Racism in the Time of COVID-19.
None CARES Act provider relief fund aid to dermatologists in response to coronavirus disease 2019 (COVID-19).

None [COVID-19 and functional dependence: cohort study of an outbreak in a nursing home for elderly.]
None Impact of the COVID-19 pandemic on human papillomavirus-based testing services to support cervical cancer screening.
None Determinants of Antimicrobial Use for Covid-19 Related Symptoms among Nigerians.
None Mitigating the Burden of Hypertension in Developing Nations and the Global Impact of Covid-19 Vaccine.
None COVID-19 Recovery Without B Cells or Antibodies in Patients Receiving Rituximab for Autoimmune Disease.
None Multi Organ Involvement in a Kidney Transplant Recipient, A Case Report.
None Let's apply the lessons from the AIDS crisis to the COVID-19 pandemic.
None [Asylum seekers : collaboration between nurses and medical students during the COVID-19 epidemic].
None [A double pandemic : the impact of anti-Covid measures on obesity].
None [Obesity and COVID-19 : Mechanisms involved in the collusion of two pandemics].
None [MEASLES VACCINATION IN CONTEMPORARY HALAKHAH].
None

None [Telemedicine: a Proxemics tool of Primary Care?]
None [Tobacco use in Spain during COVID-19 lockdown: an evaluation through social media.]
None [Administration of the immunomodulatory drug aminodihydrophthalazinedione sodium for prevention of progression pneumonia induced COVID-19].
None [Home telemonitoring and oxygen therapy in COVID-19 patients: safety, patient satisfaction, and cost-effectiveness].
None Helping Ambivalent Patients Make Healthy Decision About COVID-19.
None Genetic Characteristics and Phylogeny of 969-bp S Gene Sequence of SARS-CoV-2 from Hawai'i Reveals the Worldwide Emerging P681H Mutation.
None Rapid Autopsy Programs and Research Support: The Pre- and Post-COVID-19 Environments.
None [What is the effect of social distancing on the course of COVID-19 epidemic?]
None [Anti-inflammatory effects of amantadine and memantine in SARS-CoV-2 infection].
None Tablet-based limited echocardiography at COVID-19-dedicated hospital during the pandemic in the context of ta

HTTPError: 500 Server Error: Internal Server Error for url: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?tool=MyTool&email=my%40email.address&db=pubmed&id=33691151&id=33691149&id=33691146&id=33691145&id=33691139&id=33691138&id=33691095&id=33691094&id=33691089&id=33691088&id=33691058&id=33691036&id=33691035&id=33691029&id=33691019&id=33691011&id=33691009&id=33691008&id=33690991&id=33690946&id=33690939&id=33690932&id=33690928&id=33690910&id=33690889&id=33690864&id=33690862&id=33690854&id=33690836&id=33690835&id=33690834&id=33690830&id=33690823&id=33690812&id=33690794&id=33690789&id=33690784&id=33690782&id=33690730&id=33690722&id=33690708&id=33690704&id=33690701&id=33690700&id=33690699&id=33690695&id=33690689&id=33690685&id=33690679&id=33690672&id=33690662&id=33690641&id=33690626&id=33690607&id=33690600&id=33690595&id=33690234&id=33690208&id=33690207&id=33690206&id=33690205&id=33690204&id=33690203&id=33690202&id=33690201&id=33690200&id=33690199&id=33690156&id=33690150&id=3369

In [6]:
with open('dict_articles.pkl', 'wb') as handle:
    pickle.dump(dict_articles,handle)
with open('dict_authors.pickle', 'wb') as handle:
    pickle.dump(dict_authors,handle)


In [ ]:
### Now let's vectorize every word in our vocab and store the embedding

In [7]:
from transformers import *
import torch
import numpy as np

scibert_tokenizer = AutoTokenizer.from_pretrained('dmis-lab/biobert-v1.1')
scibert_model = AutoModel.from_pretrained('dmis-lab/biobert-v1.1')

In [15]:
dict_keywords={}
counter = 0
for key,values in dict_articles.items():
    counter+=1
    print(counter,end='\r')
    keywords = values['Keywords']
    if keywords:
        for keyword in keywords:
            if keyword not in dict_keywords:
                token_keyword = torch.tensor(scibert_tokenizer.encode(keyword)).unsqueeze(0)
                out = scibert_model(token_keyword)
                embedding = out[0][0][1:-1].detach().numpy().mean(axis=0)
                try:
                    np.isnan(np.sum(embedding))
                    dict_keywords[keyword] = embedding
                except TypeError:
                    print('TypeError',embedding)
random.sample(dict_keywords.items(),1)

<ipython-input-15-7600a710b444>:12: RuntimeWarning: Mean of empty slice.
  embedding = out[0][0][1:-1].detach().numpy().mean(axis=0)
/home/bastien/.local/lib/python3.8/site-packages/numpy/core/_methods.py:153: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


[('health chatbots',
  array([ 1.02772228e-01, -2.90360123e-01, -2.33494788e-01,  2.09113717e-01,
         -3.97124439e-02, -1.55656964e-01,  3.06366563e-01,  3.83730493e-02,
          3.09444740e-02,  2.31232762e-01,  1.52627975e-01,  2.19115227e-01,
          5.30987233e-02, -9.94683877e-02, -4.50131834e-01, -1.14051580e-01,
          1.23077296e-01,  1.23471497e-02,  3.69843543e-02,  1.07185930e-01,
         -1.54830486e-01, -4.06012535e-02,  2.74198875e-02,  5.79762608e-02,
          7.86282420e-02,  7.05272332e-03,  3.18916231e-01, -2.13626921e-01,
         -1.92025080e-01,  1.80557519e-01, -1.25880972e-01,  4.76782948e-01,
          1.10111073e-01, -2.38154270e-02,  8.66393670e-02,  7.02129444e-03,
         -1.26301616e-01, -2.41521209e-01, -5.93014583e-02,  3.22189152e-01,
          3.12419713e-01,  2.93428957e-01,  4.03830349e-01, -2.66135603e-01,
          4.77097183e-03, -1.81620821e-01, -7.84306228e-02,  2.30242312e-01,
         -1.33450523e-01,  7.64599442e-02,  2.82079488e

In [ ]:
with open('dictionary.pickle', 'wb') as handle:
    pickle.dump(dict_keywords,handle)